Name: Daniel Adejumo

Net ID: dxa239

#### 1. Actors and Directors Who Cooperated At Least Three Times 
https://leetcode.com/problems/actors-and-directors-who-cooperated-at-least-three-times/

In [ ]:
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(timestamp) >= 3;

In [ ]:
import pandas as pd

def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    # Group by actor and director, then count the occurrences
    grouped = actor_director.groupby(['actor_id', 'director_id']).size().reset_index(name='counts')
    
    # Filter where counts are at least 3
    result = grouped[grouped['counts'] >= 3]
    
    # Return only the required columns
    return result[['actor_id', 'director_id']]

#### 2. Fix Names in a Table 
https://leetcode.com/problems/fix-names-in-a-table/

In [ ]:
SELECT 
    user_id, 
    CONCAT(UPPER(SUBSTR(name, 1, 1)), LOWER(SUBSTR(name, 2))) AS name
FROM Users
ORDER BY user_id;

In [ ]:
import pandas as pd

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    # Apply the capitalize string method to the 'name' column
    users['name'] = users['name'].str.capitalize()
    
    # Sort the DataFrame by 'user_id'
    result = users.sort_values(by='user_id')
    
    # Return the result
    return result


#### 3. Combine Two Tables 
https://leetcode.com/problems/combine-two-tables/

In [ ]:
SELECT 
    Person.firstName, 
    Person.lastName, 
    Address.city, 
    Address.state
FROM 
    Person
LEFT JOIN 
    Address 
ON 
    Person.personId = Address.personId;

In [ ]:
import pandas as pd

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    # Perform a left merge on personId
    merged_df = person.merge(address, on='personId', how='left')
    
    # Select and reorder the required columns
    result = merged_df[['firstName', 'lastName', 'city', 'state']]
    
    return result

#### 4. Second Highest Salary
https://leetcode.com/problems/second-highest-salary/

In [ ]:
SELECT 
    MAX(salary) AS SecondHighestSalary
FROM 
    Employee
WHERE 
    salary < (SELECT MAX(salary) FROM Employee);

In [ ]:
import pandas as pd

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    # 1. Drop duplicate salaries to ensure we look at distinct values
    unique_salaries = employee['salary'].drop_duplicates()
    
    # 2. Sort salaries in descending order
    sorted_salaries = unique_salaries.sort_values(ascending=False)
    
    # 3. Check if a second highest salary exists
    if len(sorted_salaries) < 2:
        return pd.DataFrame({'SecondHighestSalary': [None]})
    
    # 4. Get the second salary (index 1)
    second_salary = sorted_salaries.iloc[1]
    
    return pd.DataFrame({'SecondHighestSalary': [second_salary]})


#### 5. List the Products Ordered in a Period
https://leetcode.com/problems/list-the-products-ordered-in-a-period/

In [ ]:
SELECT 
    p.product_name, 
    SUM(o.unit) AS unit
FROM 
    Products p
JOIN 
    Orders o 
ON 
    p.product_id = o.product_id
WHERE 
    o.order_date BETWEEN '2020-02-01' AND '2020-02-29'
GROUP BY 
    p.product_id, 
    p.product_name
HAVING 
    SUM(o.unit) >= 100;

In [ ]:
import pandas as pd

def list_products(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    # 1. Filter for February 2020
    feb_orders = orders[
        (orders['order_date'] >= '2020-02-01') & 
        (orders['order_date'] <= '2020-02-29')
    ]
    
    # 2. Group by product_id and sum units
    product_stats = feb_orders.groupby('product_id')['unit'].sum().reset_index()
    
    # 3. Filter for at least 100 units
    high_volume_products = product_stats[product_stats['unit'] >= 100]
    
    # 4. Merge with products table to get names
    result = high_volume_products.merge(products, on='product_id')
    
    # 5. Return only specific columns
    return result[['product_name', 'unit']]

#### 6. Replace Employee ID With The Unique Identifier
https://leetcode.com/problems/replace-employee-id-with-the-unique-identifier/

In [ ]:
SELECT 
    eu.unique_id, 
    e.name
FROM 
    Employees e
LEFT JOIN 
    EmployeeUNI eu 
ON 
    e.id = eu.id;

In [ ]:
import pandas as pd

def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    # 1. Perform a left merge on 'id'
    # This keeps all rows from 'employees' and fills missing 'employee_uni' data with NaN (null)
    merged_df = employees.merge(employee_uni, on='id', how='left')
    
    # 2. Select and reorder the required columns
    result = merged_df[['unique_id', 'name']]
    
    return result

#### 7. Game Play Analysis IV
https://leetcode.com/problems/game-play-analysis-iv/

In [ ]:
SELECT 
    ROUND(COUNT(t2.player_id) / COUNT(t1.player_id), 2) AS fraction
FROM 
    (SELECT player_id, MIN(event_date) AS first_login 
     FROM Activity 
     GROUP BY player_id) AS t1
LEFT JOIN 
    Activity AS t2 
ON 
    t1.player_id = t2.player_id 
    AND t2.event_date = DATE_ADD(t1.first_login, INTERVAL 1 DAY);


In [ ]:
import pandas as pd

def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    # 1. Find the first login date for each player
    first_logins = activity.groupby('player_id')['event_date'].min().reset_index()
    
    # 2. Calculate the target "next day" date
    first_logins['next_day'] = first_logins['event_date'] + pd.Timedelta(days=1)
    
    # 3. Merge first_logins with the original activity table
    # We want to see if (player_id, next_day) exists in the original data (player_id, event_date)
    merged = first_logins.merge(
        activity, 
        left_on=['player_id', 'next_day'], 
        right_on=['player_id', 'event_date'],
        how='inner'
    )
    
    # 4. Calculate the fraction
    # Numerator: Number of players in the merged result (those who logged in next day)
    # Denominator: Total number of unique players (length of first_logins)
    if len(first_logins) == 0:
        fraction = 0.0
    else:
        fraction = len(merged) / len(first_logins)
    
    # 5. Round to 2 decimal places and format return
    return pd.DataFrame({'fraction': [round(fraction, 2)]})

#### 8. Project Employees I
https://leetcode.com/problems/project-employees-i/

In [ ]:
SELECT 
    p.project_id, 
    ROUND(AVG(e.experience_years), 2) AS average_years
FROM 
    Project p
JOIN 
    Employee e 
ON 
    p.employee_id = e.employee_id
GROUP BY 
    p.project_id;

In [ ]:
import pandas as pd

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    # 1. Merge Project and Employee tables
    merged_df = project.merge(employee, on='employee_id', how='left')
    
    # 2. Group by project_id and calculate the mean of experience_years
    avg_experience = merged_df.groupby('project_id')['experience_years'].mean().reset_index()
    
    # 3. Round to 2 decimal places
    avg_experience['experience_years'] = avg_experience['experience_years'].round(2)
    avg_experience = avg_experience.rename(columns={'experience_years': 'average_years'})
    
    return avg_experience

#### 9. Department Top Three Salaries
https://leetcode.com/problems/department-top-three-salaries/

In [ ]:
WITH RankedSalaries AS (
    SELECT 
        d.name AS Department,
        e.name AS Employee,
        e.salary AS Salary,
        DENSE_RANK() OVER (
            PARTITION BY e.departmentId 
            ORDER BY e.salary DESC
        ) AS rnk
    FROM 
        Employee e
    JOIN 
        Department d 
    ON 
        e.departmentId = d.id
)
SELECT 
    Department, 
    Employee, 
    Salary
FROM 
    RankedSalaries
WHERE 
    rnk <= 3;


In [ ]:
import pandas as pd

def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    # 1. Merge Employee and Department tables
    # Suffixes are used to distinguish between employee name and department name
    merged_df = employee.merge(
        department, 
        left_on='departmentId', 
        right_on='id', 
        suffixes=('_emp', '_dept')
    )
    
    # 2. Calculate Dense Rank
    # Group by department and rank salaries descending
    merged_df['rank'] = merged_df.groupby('departmentId')['salary'].rank(
        method='dense', 
        ascending=False
    )
    
    # 3. Filter for top 3
    top_earners = merged_df[merged_df['rank'] <= 3]
    
    # 4. Select and Rename columns to match output format
    result = top_earners[['name_dept', 'name_emp', 'salary']]
    result.columns = ['Department', 'Employee', 'Salary']
    
    return result